In [25]:
from summary_reader import read_tensors

filename = "../results/dnn-regularized/error_analysis/events.out.tfevents.*"
iterator = read_tensors(filename)

x = next(iterator)

In [31]:
import numpy as np

# Find errors in the batch.
idx = np.where(x['is_error'].ravel())[0]

print idx
np.sum(x['is_error'])

def examples_by_index(features, indices):
    return [dict((k, features[k][idx]) for k in features.keys()) for idx in indices]

def html_table(errors, keys=None):
    """ Returns HTML string for a table of the errors.
    
    Args:
        errors: List of dictionaries. All elements of dictionaries will be represented
                as strings in the table.
        keys: Keys from the dictionaries that should be shown. If None, will use keys
              from first element of the |errors|.

    Returns:
        Valid HTML string.
    """
    if not errors:
        return ""

    keys = keys or errors[0].keys()
    s = "<table>"
    s += "<tr><th>" + "</th><th>".join(keys) + "</th></tr>"
    row_template = "<tr><td>"+"</td><td>".join("{"+key+"}" for key in keys) + "</td></tr>"
    s += "".join(row_template.format(**error) for error in errors)
    s += "</table>"
    return s

from IPython.display import HTML, display
def show_table(errors):
    display(HTML(html_table(errors)))

show_table(examples_by_index(x, idx))

[ 4  6  7 15 19 20 22 23 26 31 37 39 49 51 62 64 65 66 70 81 84 85 88 99]


BillTitle,is_error,VoterState,SponsorParty,prediction,label,VoterParty,VoterAge
[ 4 13 1 117 203 11 43 113 4 1 117 203 465 134 55 3 1 117 203 359 365 55 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['WA'],['democrat'],[ True],[0],['republican'],[76]
[12 10 2 1 23 5 39 2 1 8 9 14 15 16 80 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['LA'],['democrat'],[False],[1],['republican'],[61]
[ 4 13 38 1282 21 19 28 4 18 20 898 1424 4 173 3 472 31 17 1 1419 2226 17 1 750 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['NJ'],['democrat'],[False],[1],['republican'],[47]
[ 12 10 2 132 230 31 122 3 149 70 3 24 25 33 2 1 8 9 14 15 16 126 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['FL'],['republican'],[ True],[0],['republican'],[86]
[ 4 18 2 20 681 1754 1 5 20 3 2 1 626 5 20 37 557 1020 51 4 174 29 1397 1395 1754 1 365 5 171 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['MI'],['democrat'],[ True],[0],['democrat'],[65]
[ 4 13 1 514 177 3 81 90 11 4 26 20 55 2 4 596 1 70 5 514 177 4 101 1 22 348 5 514 90 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['CO'],['republican'],[False],[1],['democrat'],[59]
[ 4 78 1 279 443 11 5 69 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['NY'],['UNK'],[False],[1],['democrat'],[66]
[12 10 2 1 23 5 39 2 1 8 9 14 15 16 80 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['CA'],['democrat'],[False],[1],['republican'],[40]
[12 10 2 1 23 5 39 2 1 8 9 14 15 16 69 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['NJ'],['republican'],[ True],[0],['republican'],[72]
[12 92 68 10 2 1 8 9 79 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[False],['CA'],['democrat'],[ True],[0],['republican'],[86]
